<a href="https://colab.research.google.com/github/Skander28/Models/blob/main/LSTM(over%2Blem).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
  !nvidia-smi
else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4
Wed May  3 08:27:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                            

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/

'carte visite badra.gdoc'
 Classroom
'Colab Notebooks'
 dev_labeled.tsv
 django
 entreprise-1-20220417T102515Z-001.zip
'Espace citoyen.pdf'
 ez-learning
 maghrebi
'model (1).pth'
 preprocessed_tweets.csv
 pre_tweets.csv
 Projet_Java_FediBoussaada_LaribiSkander_2A
 Projet_Microservices
 trainer2.pth
 train_labeled.tsv
 tweets.csv
'Untitled presentation.gslides'
 updated_dataset.csv


In [7]:
import pandas as pd
tweets_ids = pd.read_csv('/content/drive/MyDrive/maghrebi/dialect_dataset.csv')
     

# Read the given dataset which contain the Ids and the labels
df_clean = pd.read_csv('/content/drive/MyDrive/maghrebi/messages.csv',
                 lineterminator='\n')

column_names = ['id', 'tweets']  # list of column names

df_clean.columns = column_names

tweets_dataset = pd.merge(tweets_ids, df_clean, on='id', how='inner')
     
filtered_df = tweets_dataset[tweets_dataset['dialect'].isin(['TN','DZ','MA','LY'])]
filtered_df.head()

,id,dialect,tweets
15497,1009754958479151232,LY,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو ا...
15498,1009794751548313600,LY,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ...
15499,1019989115490787200,LY,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بير...
15500,1035479791758135168,LY,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا...
15501,1035481122921164800,LY,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ا...


In [8]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
#lemmatization
lemmatizer = WordNetLemmatizer()
filtered_df['tweets'] = filtered_df['tweets'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos=wordnet.VERB) for word in x.split()]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
<ipython-input-8-3428f7852987>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['tweets'] = filtered_df['tweets'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos=wordnet.VERB) for word in x.split()]))


In [11]:
from imblearn.over_sampling import RandomOverSampler
#Oversampling
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(filtered_df.drop('dialect', axis=1), filtered_df['dialect'])
filtered_df = pd.concat([X_resampled, y_resampled], axis=1)

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


In [13]:
# Load data
features = filtered_df.tweets.values
labels = pd.get_dummies(filtered_df['dialect']).values

In [14]:
vocab_size = 20000
max_length= 200
tokenizer = Tokenizer(num_words=vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(features)
X = tokenizer.texts_to_sequences(features)


In [15]:
import nltk
from collections import Counter
nltk.download('stopwords')
from nltk.corpus import stopwords

# use the stopwords
stop_words = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
all_words = []
for tokens in X:
    all_words.extend(tokens)

stop_words = set(stopwords.words('arabic'))
all_words = [word for word in all_words if word not in stop_words]

word_counts = Counter(all_words)
most_common_words = [word for word, count in word_counts.most_common(500)]

def remove_common_words(tokens):
  new_tokens = [token for token in tokens if token not in most_common_words]
  return new_tokens

X = remove_common_words(X)


X = pad_sequences(X, maxlen=max_length)


In [17]:
# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.1, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, shuffle=True)


In [18]:
# Convert data to PyTorch tensors
X_train, y_res_ = torch.tensor(X_train), torch.tensor(y_train)
X_val, y_val = torch.tensor(X_val), torch.tensor(y_val)
X_test, y_test = torch.tensor(X_test), torch.tensor(y_test)

In [19]:
class DialectDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [20]:
# Create dataloaders for training, validation, and test sets
train_dataset = DialectDataset(X_train, y_train)
val_dataset = DialectDataset(X_val, y_val)
test_dataset = DialectDataset(X_test, y_test)

In [21]:
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


# **LSTM Model**

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np

output_dim = 100
batch_size = 64


class MyModel(nn.Module):
    def __init__(self, vocab_size, output_dim, max_length):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, output_dim)
        self.dropout = nn.Dropout(p=0.5)
        self.lstm = nn.LSTM(output_dim, 50, dropout=0.3, batch_first=True)
        self.lstm2 = nn.LSTM(50, 25, dropout=0.3, batch_first=True)
        self.linear = nn.Linear(25, 4)
        softmax = nn.Softmax(dim=1)
        self.fc = nn.Sequential(self.linear, softmax)
    
    def forward(self, x):
        embedded = self.embedding(x)
        dropped = self.dropout(embedded)
        lstm_out, _ = self.lstm(dropped)
        lstm2_out, _ = self.lstm2(lstm_out)
        out = self.fc(lstm2_out[:, -1, :])
        return out

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MyModel(vocab_size, output_dim, max_length).to(device)
optimizer = optim.Adam(model.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss()
counter_nb = 0
for epoch in tqdm(range(100)):
    model.train()
    counter_nb = counter_nb + 1 
    print(counter_nb)
    running_loss = 0.0
    with torch.cuda.device(0):
      for batch in train_dataloader:
        inputs, labels = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()
        labels = labels.float()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / (len(train_dataloader))

    best_val_loss = np.inf
    patience = 5
    counter = 0
    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for batch in val_dataloader:
            inputs, labels = batch[0].to(device), batch[1].to(device)
            labels = labels.float()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss = val_loss / (len(val_dataloader))
    # Check if the validation loss has improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1

    # Stop the training process if the validation loss hasn't improved for `patience` epochs
    if counter >= patience:
        break
    print(" epoch loss :", epoch_loss , "| val loss :", val_loss)

print("Training stopped after epoch", epoch)
        

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/100 [00:00<?, ?it/s]

1


  1%|          | 1/100 [00:17<29:35, 17.94s/it]

 epoch loss : 1.1266609331294573 | val loss : 0.9787683304652427
2


  2%|▏         | 2/100 [00:28<22:11, 13.59s/it]

...

 epoch loss : 0.8625921804732058 | val loss : 0.8660815991822955
99


 99%|█████████▉| 99/100 [17:20<00:10, 10.46s/it]

 epoch loss : 0.8626023002001114 | val loss : 0.8641132854142235
100


100%|██████████| 100/100 [17:30<00:00, 10.51s/it]

 epoch loss : 0.8633843044310937 | val loss : 0.8674208675194712
Training stopped after epoch 99


In [24]:
torch.save(model.state_dict(), 'model.pth')

# **Evaluation**

In [25]:
model = MyModel(vocab_size, output_dim, max_length)
model.load_state_dict(torch.load("/content/model.pth", map_location=torch.device('cpu')))

model.eval()
X_test = X_test.to("cpu")
# Compute the model's prediction for the padded sequence
with torch.no_grad():
    pred_ = model(X_test)

In [26]:
pred__ = pred_.cpu().numpy()
preds = np.argmax(pred__, axis=1)

In [27]:
def one_hot(a, num_classes):
  return np.squeeze(np.eye(num_classes)[a.reshape(-1)])
pred_hot = one_hot(preds,4)

In [28]:
x_np = torch.from_numpy(pred_hot)

In [29]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
recall_score(y_test, x_np,average='macro') 

0.8764814400615528

In [30]:
precision_score(y_test, x_np,average='macro') 

0.8765281955384712

In [31]:
accuracy_score(y_test, pred_hot)

0.8769178082191781

In [32]:
f1_score(y_test, x_np,average='macro')

0.8764875174109238

# **Testing**

In [33]:
# Set the model to evaluation mode
model.eval()
device = "cuda"
# Tokenize the new complaint and pad the sequence
new_complaint = ['واش دير للعشى']
#new_complaint  = ["شوكران علا هاد "]
#new_complaint  = ["نبي نروح للحوش"]
#new_complaint  = ["شبيك شتحب "]
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=max_length)

# Convert the padded sequence to a PyTorch tensor and move it to the device (e.g., GPU) if available
padded_tensor = torch.LongTensor(padded).to("cpu")

# Compute the model's prediction for the padded sequence
with torch.no_grad():
    pred = model(padded_tensor)

# Move the prediction back to the CPU and convert to a numpy array
pred = pred.cpu().numpy()

# Map the prediction to a class label using the CLASS_DICT{'DZ': 0, 'LY': 1, 'MA': 2, 'TN': 3}
CLASS_DICT = {1: "LY", 3: "TN", 2: "MA", 0: "DZ" }
class_label = CLASS_DICT[np.argmax(pred)]

# Print the prediction and the predicted class label
print(class_label)

DZ
